In [1]:
import pandas as pd
import numpy as np
url = 'dr_who_dataset.csv'
df_original = pd.read_csv(url,index_col=0,parse_dates=[0])

In [2]:
dr_who = df_original.copy()
dr_who=dr_who.convert_dtypes()
#dr_who.dtypes

In [3]:
from imdb import IMDb
from datetime import datetime
# create an instance of the IMDb class
ia = IMDb()

### Data Transformation

In [4]:
dr_who['Episode Title'] = dr_who['Episode Title'].str.strip('(missing)')
dr_who['Episode Title'] = dr_who['Episode Title'].fillna('')
dr_who['Serial title'] = dr_who['Serial title'].fillna('')

In [5]:
dr_who.loc[(dr_who['Serial title'] == 'The Space Pirates') &
            (dr_who['Episode Title'] =='\"Episode One\"'),'Episode Title']='"Episode 1"'

dr_who.loc[(dr_who['Serial title'] == 'The Space Pirates') &
            (dr_who['Episode Title'] =='\"Episode Three\"'),'Episode Title']='"Episode 3"'

dr_who.loc[(dr_who['Serial title'] == 'The Space Pirates') &
            (dr_who['Episode Title'] =='\"Episode Four\"'),'Episode Title']='"Episode 4"'

dr_who.loc[(dr_who['Serial title'] == 'The Space Pirates') &
            (dr_who['Episode Title'] =='\"Episode Five\"'),'Episode Title']='"Episode 5"'

dr_who.loc[(dr_who['Serial title'] == 'The Space Pirates') &
            (dr_who['Episode Title'] =='\"Episode Six\"'),'Episode Title']='"Episode 6"'

dr_who.loc[(dr_who['Serial title'] == 'The Abominable Snowmen') &
            (dr_who['Episode Title'] =='\"Episode One\"'),'Episode Title']='"Episode 1"'

dr_who.loc[(dr_who['Episode Title'] =='"Spyfall, Part 1"'),'Episode Title']='"Spyfall, Part One"'

dr_who.loc[(dr_who['Episode Title'] =='"Spyfall, Part 2"'),'Episode Title']='"Spyfall, Part Two"'

### Get IMDb ID

In [6]:
#pip install tqdm
from tqdm.notebook import tqdm, trange
import time    # to be used in loop iterations

In [7]:
def get_imdb_ids(df, id_list):
    with tqdm(total=len(df)) as p_bar:
        for index, row in df.iterrows():
            dt = datetime.strptime(row['Original air date'], '%Y-%m-%d')
            p_bar.set_description(f'Working on "{index}"')
            p_bar.update(1)
            title = ('Doctor Who '+ row['Serial title'] +' '+ row['Episode Title'] +' '+ str(dt.year))
            episode = ia.search_episode(title)
            if len(episode) == 0:
                title = ('Doctor Who ' + row['Episode Title'] + str(dt.year))
                episode = ia.search_episode(title)
            temp_list = []
            for e in range(0,len(episode)):
                if episode[e]['year'] == dt.year and episode[e]['episode of']=='Doctor Who':
                    ide = episode[e].movieID
                    temp_list.append(ide)
            if len(temp_list) >= 1:
                ide = temp_list[0]
                id_list.append(ide)

In [8]:
imdb_ids = []
get_imdb_ids(dr_who,imdb_ids)
#len(imdb_ids)

### Companions and Other Characters 

#### Companions

In [9]:
cast_doctor = ['William Hartnell','Patrick Troughton','Jon Pertwee','Tom Baker','Peter Davison','Colin Baker','Sylvester McCoy',
         'Christopher Eccleston','David Tennant','Matt Smith','Peter Capaldi','Jodie Whittaker']
cast_companions = ['Carole Ann Ford','Jacqueline Hill','William Russell','Maureen O\'Brien','Peter Purves','Adrienne Hill',
                   'Jean Marsh','Jackie Lane','Anneke Wills','Michael Craze','Frazer Hines','Deborah Watling','Wendy Padbury',
                   'Nicholas Courtney','Caroline John','Katy Manning','Elisabeth Sladen','John Levene','Richard Franklin',
                   'Ian Marter','Louise Jameson','John Leeson', 'David Brierly','Mary Tamm','Lalla Ward','Matthew Waterhouse',
                   'Sarah Sutton','Janet Fielding','Mark Strickson','Gerald Flood','Nicola Bryant','Bonnie Langford',
                   'Sophie Aldred','Daphne Ashbrook','Billie Piper','Bruno Langley','John Barrowman','Noel Clarke',
                   'Catherine Tate','Freema Agyeman','Kylie Minogue','David Morrissey','Michelle Ryan','Adelaide Brooke',
                   'Bernard Cribbins','Karen Gillan','Arthur Darvill','Alex Kingston','James Corden','Jenna Coleman',
                   'Matt Lucas','Pearl Mackie','Bradley Walsh','Tosin Cole','Mandip Gill']

In [10]:
def doctors_companions(ids, companions_list,split_companions,split_id, split_cast):
    with tqdm(total=len(ids)) as p_bar:
        for i in ids:
            p_bar.set_description(f'Working on "{i}"')
            p_bar.update(1)
            episode = ia.get_movie_main(i)
            actors = episode['data']['cast']
            title = episode['data']['title']
            temp_list = []
            for n in range(len(actors)):
                try:
                    cast = actors[n]
                    name = cast['name']
                    role = str(cast.currentRole['name'])
                    for c in cast_companions:
                        if name == c:
                            temp_list.append(role)
                            split_companions.append(role)
                            split_id.append(i)
                            split_cast.append(name)
                except:
                    pass
            companions_list.append(temp_list)

In [11]:
companions = []
split_companions = []
split_comp_id = []
split_comp_cast = []
doctors_companions(imdb_ids, companions,split_companions, split_comp_id,split_comp_cast)

In [12]:
df_companions = pd.DataFrame()
df_companions['IMDb Id'] = split_comp_id
df_companions['Companion(split)'] = split_companions
df_companions['Actor'] = split_comp_cast
df_companions.to_csv('dr_who_companions.csv')

#### Adversaries and Others

In [13]:
def doctors_adversaries(ids, adversaries_list, split_adversaries,split_id):
    with tqdm(total=len(ids)) as p_bar:
        for i in ids:
            p_bar.set_description(f'Working on "{i}"')
            p_bar.update(1)
            episode = ia.get_movie_main(i)
            actors = episode['data']['cast']
            temp_list = []
            for n in range(len(actors)):
                try:
                    cast = actors[n]
                    name = cast['name']
                    role = str(cast.currentRole['name'])
                    if (name not in cast_companions and name not in cast_doctor):
                            if role not in temp_list:
                                temp_list.append(role)
                                split_adversaries.append(role)
                                split_id.append(i)
                except:
                    pass
            adversaries_list.append(temp_list)

In [14]:
adversaries = []
split_adversaries = []
split_adv_id = []
doctors_adversaries(imdb_ids, adversaries,split_adversaries, split_adv_id)

In [15]:
df_adversaries = pd.DataFrame()
df_adversaries['IMDb Id'] = split_adv_id
df_adversaries['Adversaries/Others(split)'] = split_adversaries
df_adversaries.to_csv('dr_who_adversaries_others.csv')

### Episode Plot

In [16]:
def episode_plot(ids, plot_list):
     with tqdm(total=len(ids)) as p_bar:
            for i in ids:
                p_bar.set_description(f'Working on "{i}"')
                p_bar.update(1)
                try:
                    episode = ia.get_movie_main(i)
                    ep_plot = episode['data']['plot outline']
                except KeyError:
                    try:
                        episode = ia.get_movie_plot(i)
                        ep_plot = episode['data']['plot'][0]
                    except:
                        ep_plot = 'Plot does not exist or missing.'
                plot_list.append(ep_plot)

In [17]:
imdb_plot = []
episode_plot(imdb_ids, imdb_plot)

### Episode Runtime, Rating, Sound, Color, IMDb Votes, and MDb Keywords

In [18]:
def episode_information(ids, runtime, rating, votes, sound, color, keywords):
    with tqdm(total=len(ids)) as p_bar:
        for i in ids:
            p_bar.set_description(f'Working on "{i}"')
            p_bar.update(1)
            episode = ia.get_movie_main(i)
            #Runtime
            try:
                ep_runtime = episode['data']['runtimes'][0]
            except:
                ep_runtime = 'N/A'
            #Rating
            try:
                ep_rating = episode['data']['rating']
            except:
                ep_rating = 0
            #Votes
            try:
                ep_votes = episode['data']['votes']
            except:
                ep_votes = 0
            #Sound
            try:
                ep_sound = episode['data']['sound mix'][0]
            except:
                ep_sound = 'N/A'
            #Color
            try:
                ep_color = episode['data']['color info'][0]
            except:
                ep_color = 'N/A'
            #Keywords
            try:
                words = ia.get_movie_keywords(i)
                ep_keywords = words['data']['keywords']
            except:
                ep_keywords = 'None/Missing'
                
            #Appending data
            runtime.append(ep_runtime)
            rating.append(ep_rating)
            votes.append(ep_votes)
            sound.append(ep_sound)                        
            color.append(ep_color)
            keywords.append(ep_keywords)

In [19]:
imdb_runtime = []
imdb_color =[]
imdb_sound = []
imdb_rating = []
imdb_votes = []
imdb_keywords = []
episode_information(imdb_ids, imdb_runtime, imdb_rating, imdb_votes, imdb_sound, imdb_color, imdb_keywords)

### IMDb Dataframe 

In [20]:
df_imdb = pd.DataFrame()
df_imdb['IMDb Id'] = imdb_ids
df_imdb['Plot'] = imdb_plot
df_imdb['Companions'] = companions
df_imdb['Adversaries/Others'] = adversaries
df_imdb['Episode Runtime'] = imdb_runtime
df_imdb['Rating'] = imdb_rating
df_imdb['Votes'] = imdb_votes
df_imdb['Sound Mix'] = imdb_sound
df_imdb['Episode Color'] = imdb_color
df_imdb['Episode Keywords'] = imdb_keywords
df_imdb.index = np.arange(1,len(df_imdb)+1)
df_imdb

,IMDb Id,Plot,Companions,Adversaries/Others,Episode Runtime,Rating,Votes,Sound Mix,Episode Color,Episode Keywords
1,0562828,"Two schoolteachers, Ian Chesterton and Barbara...","[Ian Chesterton, Barbara Wright, Susan Foreman]","[Shadow, Schoolgirl, Policeman, Schoolboy]",23,8.3,1477,Mono,Black and White,"[tardis, first-doctor, teenager, ignorance, cu..."
2,0782279,The TARDIS takes its new passengers back to th...,"[Ian Chesterton, Barbara Wright, Susan Foreman]","[Za, Hur, Old Mother, Kal, Horg, Tribesman, Me...",25,6.5,1043,Mono,Black and White,"[first-doctor, caveman, fire, tribe, worrying,..."
3,0805109,Old Mother frees the Doctor and the others on ...,"[Ian Chesterton, Barbara Wright, Susan Foreman]","[Old Mother, Hur, Za, Kal, Horg, Child]",24,6.4,946,Mono,Black and White,"[tribe, boulder, first-doctor, wound, grandfat..."
4,0813250,Kal accuses Za of Old Mother's murder but the ...,"[Ian Chesterton, Barbara Wright, Susan Foreman]","[Horg, Kal, Hur, Za]",24,6.6,924,Mono,Black and White,"[fire, first-doctor, science-teacher, history-..."
5,0562905,The TARDIS lands in a petrified forest on an a...,"[Ian Chesterton, Barbara Wright, Susan Foreman]",[],24,7.8,935,Mono,Black and White,"[dalek, petrified-forest, doctor, city, first-..."
...,...,...,...,...,...,...,...,...,...,...
857,9361014,"From ancient Syria to present day Sheffield, a...","[Graham O'Brien, Yasmin Khan, Ryan Sinclair]","[Zellin, Tibo, Rakaya, Grace O'Brien, Sonya Kh...",49,5.6,3089,N/A,Color,"[aleppo-syria, sheffield, year-2020, 1380s, fo..."
858,9361038,"Villa Diodati, 1816 - on a night that inspired...","[Graham O'Brien, Ryan Sinclair, Yasmin Khan]","[Mary Wollstonecraft Godwin, Lord Byron, Clair...",49,7.0,3162,N/A,Color,"[nineteenth-century, famous-author, year-1816,..."
859,9361042,"In the far future, the Doctor and her friends ...","[Graham O'Brien, Yasmin Khan, Ryan Sinclair]","[The Master, Ashad, Ravio, Ko Sharmus, Yedlarm...",49,6.4,2974,N/A,Color,"[cyberman, far-future, four-word-episode-title]"
860,9361046,"In the epic and emotional season finale, the C...","[Graham O'Brien, Ryan Sinclair, Yasmin Khan, C...","[The Master, Ashad, Ko Sharmus, Ravio, Yedlarm...",66,5.1,4150,N/A,Color,"[thirteenth-doctor, far-future, matrix, regene..."


### Final Dataframe 

In [21]:
df_final = pd.concat([df_original, df_imdb], axis=1)

In [22]:
df_final

,Story,Serial,Episode,Serial title,Episode Title,Directed by,Written by,Original air date,Prod.code,UK viewers(millions),...,IMDb Id,Plot,Companions,Adversaries/Others,Episode Runtime,Rating,Votes,Sound Mix,Episode Color,Episode Keywords
1,1,1.0,1,An Unearthly Child,"""An Unearthly Child""",Waris Hussein,Anthony Coburn and C. E. Webber (uncredited),1963-11-23,A,4.40,...,0562828,"Two schoolteachers, Ian Chesterton and Barbara...","[Ian Chesterton, Barbara Wright, Susan Foreman]","[Shadow, Schoolgirl, Policeman, Schoolboy]",23,8.3,1477,Mono,Black and White,"[tardis, first-doctor, teenager, ignorance, cu..."
2,1,1.0,2,An Unearthly Child,"""The Cave of Skulls""",Waris Hussein,Anthony Coburn,1963-11-30,A,5.90,...,0782279,The TARDIS takes its new passengers back to th...,"[Ian Chesterton, Barbara Wright, Susan Foreman]","[Za, Hur, Old Mother, Kal, Horg, Tribesman, Me...",25,6.5,1043,Mono,Black and White,"[first-doctor, caveman, fire, tribe, worrying,..."
3,1,1.0,3,An Unearthly Child,"""The Forest of Fear""",Waris Hussein,Anthony Coburn,1963-12-07,A,6.90,...,0805109,Old Mother frees the Doctor and the others on ...,"[Ian Chesterton, Barbara Wright, Susan Foreman]","[Old Mother, Hur, Za, Kal, Horg, Child]",24,6.4,946,Mono,Black and White,"[tribe, boulder, first-doctor, wound, grandfat..."
4,1,1.0,4,An Unearthly Child,"""The Firemaker""",Waris Hussein,Anthony Coburn,1963-12-14,A,6.40,...,0813250,Kal accuses Za of Old Mother's murder but the ...,"[Ian Chesterton, Barbara Wright, Susan Foreman]","[Horg, Kal, Hur, Za]",24,6.6,924,Mono,Black and White,"[fire, first-doctor, science-teacher, history-..."
5,2,2.0,5,The Daleks,"""The Dead Planet""",Christopher Barry,Terry Nation,1963-12-21,B,6.90,...,0562905,The TARDIS lands in a petrified forest on an a...,"[Ian Chesterton, Barbara Wright, Susan Foreman]",[],24,7.8,935,Mono,Black and White,"[dalek, petrified-forest, doctor, city, first-..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
857,293,NaN,7,NaN,"""Can You Hear Me?""",Emma Sullivan,Charlene James and Chris Chibnall,2020-02-09,None,4.90,...,9361014,"From ancient Syria to present day Sheffield, a...","[Graham O'Brien, Yasmin Khan, Ryan Sinclair]","[Zellin, Tibo, Rakaya, Grace O'Brien, Sonya Kh...",49,5.6,3089,N/A,Color,"[aleppo-syria, sheffield, year-2020, 1380s, fo..."
858,294,NaN,8,NaN,"""The Haunting of Villa Diodati""",Emma Sullivan,Maxine Alderton,2020-02-16,None,5.07,...,9361038,"Villa Diodati, 1816 - on a night that inspired...","[Graham O'Brien, Ryan Sinclair, Yasmin Khan]","[Mary Wollstonecraft Godwin, Lord Byron, Clair...",49,7.0,3162,N/A,Color,"[nineteenth-century, famous-author, year-1816,..."
859,295,NaN,9,NaN,"""Ascension of the Cybermen""",Jamie Magnus Stone,Chris Chibnall,2020-02-23,None,4.99,...,9361042,"In the far future, the Doctor and her friends ...","[Graham O'Brien, Yasmin Khan, Ryan Sinclair]","[The Master, Ashad, Ravio, Ko Sharmus, Yedlarm...",49,6.4,2974,N/A,Color,"[cyberman, far-future, four-word-episode-title]"
860,295,NaN,10,NaN,"""The Timeless Children""",Jamie Magnus Stone,Chris Chibnall,2020-03-01,None,4.69,...,9361046,"In the epic and emotional season finale, the C...","[Graham O'Brien, Ryan Sinclair, Yasmin Khan, C...","[The Master, Ashad, Ko Sharmus, Ravio, Yedlarm...",66,5.1,4150,N/A,Color,"[thirteenth-doctor, far-future, matrix, regene..."


In [23]:
df_final.to_csv('dr_who_extended.csv')